# Fossil fuel imports and exports to South Africa, Nigeria and Egypt
This notebook calculates the imports and exports of fossil fuels from named countries, using data from https://comtrade.un.org/data. We use the designation "from" to indicate data from the country we are considering, and "to" to reference data originating from other countries - this should not be confused with import and export data, which are found in both datasets. 

Note that references to US are using a conversion database that originated in the USA, not simply accounting for trade between the USA and the country in question. 

In [1]:
import numpy as np
import pandas as pd
from os.path import join

In [2]:
# Read data
input_folder = "input_data"
output_folder = "output"
plastic_file = "world_plastic_trade.csv"
file_to = "AllTradeTo3Countries.csv"
file_from = "AllTradeFrom3Countries.csv"
input_to = pd.read_csv(join(input_folder, file_to))
input_from = pd.read_csv(join(input_folder, file_from))
plastic_data = pd.read_csv(join(input_folder, plastic_file))
# pd.set_eng_float_format(accuracy=4)

Intial work indicated that data from 2018 seems to be more balanced than data from 2019 in terms of having both sides of the import and export by countries that should be in a reciprocal relationship. We will use that henceforth. 2020 is presumably anomalous. 

In [3]:
year = 2018
countries = ["South Africa", "Nigeria", "Egypt"]
impex = ["Import", "Export"]
plastic_waste = ["Waste, parings and scrap, of plastics", "Waste, parings and scrap, of plastics."]
report_cols = ["Reporter", "Trade Flow", "Netweight (kg)", "Infilledkg"]
composite = ["Coal; briquettes, ovoids and similar solid fuels manufactured from coal"]
crude = "Petroleum oils and oils obtained from bituminous minerals; crude"
redundant_cols = [
    "Classification", "Period", "Period Desc.", "Reporter Code", 
    "FOB Trade Value (US$)", "CIF Trade Value (US$)", 'Mode of Transport Code',
    "Alt Qty", "Alt Qty Unit Code", "Alt Qty Unit", 'Qty Unit Code', 
    "Customs", "Mode of Transport", "Customs Proc. Code", "Gross weight (kg)",
    "Qty Unit", "Qty"
]
from_year = input_from[input_from.Year==year][[col for col in input_from.columns if col not in redundant_cols]]
to_year = input_to[input_to.Year==year][[col for col in input_from.columns if col not in redundant_cols]]
plastic_year = plastic_data[plastic_data.Year==year][[col for col in plastic_data if col not in redundant_cols]]
assert set(from_year.columns) == set(to_year.columns)

In [4]:
plastic_year[["Reporter", "Trade Flow"]]

,Reporter,Trade Flow
217,Andorra,Import
218,Andorra,Export
219,Angola,Import
220,Angola,Export
221,Azerbaijan,Import
...,...,...
642,Sao Tome and Principe,Import
643,Sao Tome and Principe,Export
648,Barbados,Import
649,Barbados,Export


In [5]:
# Calculate the geometric mean, using only values that are within two orders of magnitude
def geomean_trunc(x):
    y = np.log(x[np.isfinite(x) & (x<100) & (x>0.01)])
    return np.exp(np.mean(y))
def geomean(x):
    y = np.log(x[(x != 0) & np.isfinite(x)])
    return np.exp(np.mean(y))
def logstdev_trunc(x):
    y = np.log(x[np.isfinite(x) & (x<100) & (x>0.01)])
    return np.exp(np.std(y))

In [6]:
plastic_year["ValueRatio"] = plastic_year["Trade Value (US$)"] / plastic_year["Netweight (kg)"]

In [7]:
plastic_year

,Year,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,2nd Partner Code,2nd Partner,2nd Partner ISO,Commodity Code,Commodity,Netweight (kg),Trade Value (US$),Flag,ValueRatio
217,2018,4,0,1,Import,Andorra,AND,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",5.0,21,0,4.200000
218,2018,4,0,2,Export,Andorra,AND,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",1061790.0,260004,0,0.244873
219,2018,4,0,1,Import,Angola,AGO,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",NaN,2155813,6,NaN
220,2018,4,0,2,Export,Angola,AGO,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",272855.0,133902,6,0.490744
221,2018,4,0,1,Import,Azerbaijan,AZE,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",4041242.0,1647736,0,0.407730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,2018,4,0,1,Import,Sao Tome and Principe,STP,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",1040.0,4222,0,4.059615
643,2018,4,0,2,Export,Sao Tome and Principe,STP,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",142.0,160,0,1.126761
648,2018,4,0,1,Import,Barbados,BRB,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",2066.0,11339,0,5.488383
649,2018,4,0,2,Export,Barbados,BRB,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",772446.0,134558,0,0.174197


We will infill the missing values in the table. We will use the geometric mean of the values in the table, since the values don't range over too large a range of values.

In [8]:
geomean(plastic_year["ValueRatio"][plastic_year["Commodity"].isin(plastic_waste)])

0.4149051392913076

In [9]:
geomean_trunc(plastic_year["ValueRatio"][plastic_year["Commodity"].isin(plastic_waste)])

0.4057896007508648

In [10]:
min(plastic_year["ValueRatio"][plastic_year["Commodity"].isin(plastic_waste)])

0.01425

In [11]:
ikg = "Infilledkg"
plastic_year[ikg] = plastic_year["Netweight (kg)"]
infill0 = ((plastic_year[ikg]==0) | np.isnan(plastic_year["Infilledkg"])) & (plastic_year["Commodity"].isin(plastic_waste))
plastic_ratio = geomean(plastic_year["ValueRatio"][plastic_year["Commodity"].isin(plastic_waste)])
plastic_year[ikg].loc[infill0] = plastic_year["Trade Value (US$)"]*plastic_ratio
plastic_year[plastic_year["Commodity"].isin(plastic_waste)]

c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Year,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,2nd Partner Code,2nd Partner,2nd Partner ISO,Commodity Code,Commodity,Netweight (kg),Trade Value (US$),Flag,ValueRatio,Infilledkg
217,2018,4,0,1,Import,Andorra,AND,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",5.0,21,0,4.200000,5.000000e+00
218,2018,4,0,2,Export,Andorra,AND,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",1061790.0,260004,0,0.244873,1.061790e+06
219,2018,4,0,1,Import,Angola,AGO,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",NaN,2155813,6,NaN,8.944579e+05
220,2018,4,0,2,Export,Angola,AGO,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",272855.0,133902,6,0.490744,2.728550e+05
221,2018,4,0,1,Import,Azerbaijan,AZE,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics",4041242.0,1647736,0,0.407730,4.041242e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,2018,4,0,1,Import,Sao Tome and Principe,STP,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",1040.0,4222,0,4.059615,1.040000e+03
643,2018,4,0,2,Export,Sao Tome and Principe,STP,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",142.0,160,0,1.126761,1.420000e+02
648,2018,4,0,1,Import,Barbados,BRB,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",2066.0,11339,0,5.488383,2.066000e+03
649,2018,4,0,2,Export,Barbados,BRB,0,World,WLD,NaN,NaN,NaN,3915,"Waste, parings and scrap, of plastics.",772446.0,134558,0,0.174197,7.724460e+05


In [12]:
big_plastic_report = plastic_year[
    plastic_year["Trade Flow"].isin(impex) & plastic_year.Commodity.isin(plastic_waste)
][report_cols]
big_plastic_report_recon = big_plastic_report.pivot(columns="Trade Flow", index=["Reporter"], values=ikg).copy()

In [13]:
big_plastic_report = big_plastic_report.pivot(columns="Trade Flow", index=["Reporter"], values="Netweight (kg)")
big_plastic_report["Net Import"] = big_plastic_report.Import - big_plastic_report.Export
big_plastic_report = big_plastic_report.sort_values(by="Import", ascending=False)

In [14]:
big_plastic_report_recon.head(10)

Trade Flow,Export,Import
Reporter,,
Andorra,1061790.0,5.000000e+00
Angola,272855.0,8.944579e+05
Antigua and Barbuda,16081.0,NaN
Argentina,NaN,3.901000e+04
Armenia,169382.0,4.000400e+04
Aruba,NaN,1.000000e+00
Australia,153831432.0,1.325389e+07
Austria,177370629.0,1.989346e+08
Azerbaijan,19000.0,4.041242e+06


In [15]:
big_plastic_report_recon["Net Import"] = big_plastic_report_recon.Import - big_plastic_report_recon.Export
big_plastic_report_recon = big_plastic_report_recon.sort_values(by="Import", ascending=False)
# We divide by 1000000 to convert to thousand tonnes
big_plastic_report_recon.head(10) / 1000000

Trade Flow,Export,Import,Net Import
Reporter,,,
Malaysia,46.114685,872.530652,826.415967
"China, Hong Kong SAR",287.764630,598.046363,310.281733
Netherlands,341.791027,555.419225,213.628198
Thailand,190.619833,552.726632,362.106799
Germany,1048.687294,468.022146,-580.665148
USA,185.243890,442.291077,257.047187
Turkey,15.643011,436.909935,421.266924
"Other Asia, nes",60.824002,429.192961,368.368959
Indonesia,98.516368,320.451809,221.935441


In [16]:
plastic_report = big_plastic_report[big_plastic_report.index.isin(countries)]
plastic_report_reconstructed = big_plastic_report_recon[big_plastic_report_recon.index.isin(countries)]
# We divide by 1000000 to convert to thousand tonnes
plastic_report / 1000000

Trade Flow,Export,Import,Net Import
Reporter,,,
Egypt,0.002999,19.889853,19.886854
South Africa,11.389890,18.074903,6.685013
Nigeria,2.164750,16.612038,14.447288


We can compare these to the data found in https://www.statista.com/chart/14383/countries-importing-plastic-waste/ for the 30 year period. The approximate ordering and order of magnitude is right. 

# Investigate fossil fuels

In [17]:
# Correct for the values of imports with known financial value but unknown weight
# Some data reports a single kilogram of goods transferred. 
# This is presumably an accounting error, and is safer to ignore than to use
nw = "Netweight (kg)"
tv = "Trade Value (US$)"
commodity_ratio = pd.DataFrame(columns=("Commodity", "ImpExp", "FromTo", "Value"))
from_year["Infilled"] = False
to_year["Infilled"] = False
for commodity in from_year.Commodity.unique():
    exp_from_ind = ((from_year["Trade Flow"] == "Export") & (from_year.Commodity == commodity)) & (from_year[nw] != 1)
    exp_to_ind = (to_year["Trade Flow"] == "Export") & (to_year.Commodity == commodity) & (to_year[nw] != 1)
    imp_from_ind = (from_year["Trade Flow"] == "Import") & (from_year.Commodity == commodity) & (from_year[nw] != 1)
    imp_to_ind = (to_year["Trade Flow"] == "Import") & (to_year.Commodity == commodity) & (to_year[nw] != 1)
    all_from_ind = (from_year.Commodity == commodity) & (from_year[nw] != 1)
    all_to_ind = (to_year.Commodity == commodity) & (to_year[nw] != 1)
    fromrat = geomean(from_year[all_from_ind]["Trade Value (US$)"] / from_year[all_from_ind][nw])
    torat = geomean(to_year[all_to_ind]["Trade Value (US$)"] / to_year[all_to_ind][nw])
    commodity_ratio = commodity_ratio.append(pd.DataFrame({
        "Commodity": [commodity, commodity, commodity, commodity, commodity, commodity],
        "ImpExp": ["Export", "Export", "Import", "Import", "Both", "Both"],
        "FromTo": ["From", "To", "From", "To", "From", "To"],
        "Value": [
            geomean(from_year[exp_from_ind]["Trade Value (US$)"] / from_year[exp_from_ind][nw]),
            geomean(to_year[exp_to_ind]["Trade Value (US$)"] / to_year[exp_to_ind][nw]),
            geomean(from_year[imp_from_ind]["Trade Value (US$)"] / from_year[imp_from_ind][nw]),
            geomean(to_year[imp_to_ind]["Trade Value (US$)"] / to_year[imp_to_ind][nw]),
            fromrat,
            torat
        ]
    }))
    infillindfrom = (from_year.Commodity == commodity) & ((from_year[nw] == 0) | np.isnan(from_year[nw]))
    infillindto = (to_year.Commodity == commodity) & ((to_year[nw] == 0) | np.isnan(to_year[nw]))
    from_year[nw].loc[infillindfrom] = from_year[tv][infillindfrom] / fromrat
    from_year["Infilled"].loc[infillindfrom] = True
    to_year[nw].loc[infillindto] = to_year[tv][infillindto] / torat
    to_year["Infilled"].loc[infillindto] = True

c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

In [18]:
# To reconstruct weights in the database we used the the average of the from values
commodity_ratio[(commodity_ratio.FromTo=="From") & (commodity_ratio.ImpExp == "Both")][["Commodity", "Value"]]

,Commodity,Value
4,"Coal; briquettes, ovoids and similar solid fue...",0.188220
4,"Coal; anthracite, whether or not pulverised, b...",0.303490
4,"Coal; bituminous, whether or not pulverised, b...",0.094687
4,"Coal; (other than anthracite and bituminous), ...",0.294514
4,"Briquettes, ovoids and similar solid fuels; ma...",0.337496
4,"Lignite; whether or not agglomerated, excludin...",0.944291
4,"Peat; (including peat litter), whether or not ...",0.881551
4,"Coke and semi-coke; of coal, lignite or peat, ...",0.381095
4,"Coal gas, water gas, producer gas and similar ...",5.768385
4,"Tar distilled from coal, from lignite, peat an...",0.992362


In [19]:
world_sum_exp = pd.DataFrame(columns=("Country","Commodity", "ExportsFrom", "World", "ImportsTo", "ReExportsFrom"))
world_sum_imp = pd.DataFrame(columns=("Country","Commodity", "ImportsFrom", "World", "ReImportsFrom", "ExportsTo"))
for commodity in from_year.Commodity.unique():
    for country in countries:
        exports_from = from_year[(from_year["Trade Flow"] == "Export") & (from_year.Commodity == commodity) & (from_year.Reporter == country)]
        reexports_from = from_year[(from_year["Trade Flow"] == "Re-Export") & (from_year.Commodity == commodity) & (from_year.Reporter == country)]
        reimports_from = from_year[(from_year["Trade Flow"] == "Re-Import") & (from_year.Commodity == commodity) & (from_year.Reporter == country)]
        imports_to = to_year[(to_year["Trade Flow"] == "Import") & (to_year.Commodity == commodity) & (to_year.Partner == country)]
        exports_to = to_year[(to_year["Trade Flow"] == "Export") & (to_year.Commodity == commodity) & (to_year.Partner == country)]
        imports_from = from_year[(from_year["Trade Flow"] == "Import") & (from_year.Commodity == commodity) & (from_year.Reporter == country)]
        assert len(exports_from[exports_from.Partner=="World"][nw]) in [0, 1]
        world_sum_exp = world_sum_exp.append(pd.DataFrame({
            "Country": country,
            "Commodity": commodity, 
            "ExportsFrom": np.nansum(exports_from[exports_from.Partner!="World"][nw]),
            "World": exports_from[exports_from.Partner=="World"][nw],
            "ImportsTo": np.nansum(imports_to[imports_to.Partner!="World"][nw]), 
            "ReExportsFrom": np.nansum(reexports_from[reexports_from.Partner!="World"][nw]) 
        }))
        assert len(imports_from[imports_from.Partner=="World"][nw]) in [0, 1]
        world_sum_imp = world_sum_imp.append(pd.DataFrame({
            "Country": country,
            "Commodity": commodity, 
            "ImportsFrom": np.nansum(imports_from[imports_from.Partner!="World"][nw]), 
            "ReImportsFrom": np.nansum(reimports_from[reimports_from.Partner!="World"][nw]), 
            "World": np.nansum(imports_from[imports_from.Partner=="World"][nw]),
            "ExportsTo": np.nansum(exports_to[exports_to.Partner!="World"][nw])
        }, index=[1]))

In [20]:
# We need to avoid a zero division error
world_sum_imp["ImpFromExpToRatio"] = world_sum_imp.ImportsFrom / (world_sum_imp.ExportsTo + 1e-20)

In [21]:
world_sum_imp

,Country,Commodity,ImportsFrom,World,ReImportsFrom,ExportsTo,ImpFromExpToRatio
1,South Africa,"Coal; briquettes, ovoids and similar solid fue...",3.051784e+09,3.051784e+09,300727000.0,1.878435e+09,1.624642e+00
1,Nigeria,"Coal; briquettes, ovoids and similar solid fue...",5.967445e+07,3.895826e+07,0.0,9.372461e+07,6.366999e-01
1,Egypt,"Coal; briquettes, ovoids and similar solid fue...",2.294180e+09,1.860907e+09,0.0,5.597427e+09,4.098633e-01
1,South Africa,"Coal; anthracite, whether or not pulverised, b...",3.542647e+08,3.542647e+08,0.0,3.681602e+07,9.622568e+00
1,Nigeria,"Coal; anthracite, whether or not pulverised, b...",1.887503e+06,1.887503e+06,0.0,1.687869e+06,1.118276e+00
1,Egypt,"Coal; anthracite, whether or not pulverised, b...",1.269627e+08,8.766593e+07,0.0,2.122845e+07,5.980782e+00
1,South Africa,"Coal; bituminous, whether or not pulverised, b...",1.273978e+09,1.273978e+09,0.0,1.470915e+09,8.661127e-01
1,Nigeria,"Coal; bituminous, whether or not pulverised, b...",0.000000e+00,0.000000e+00,0.0,9.203600e+07,0.000000e+00
1,Egypt,"Coal; bituminous, whether or not pulverised, b...",1.846916e+09,2.715053e+09,0.0,5.545267e+09,3.330617e-01
1,South Africa,"Coal; (other than anthracite and bituminous), ...",1.379110e+09,1.379110e+09,300727000.0,3.501910e+08,3.938164e+00


In [22]:
world_sum_exp["ExpFromImpToRatio"] = world_sum_exp.ExportsFrom / world_sum_exp.ImportsTo

In [23]:
world_sum_exp

,Country,Commodity,ExportsFrom,World,ImportsTo,ReExportsFrom,ExpFromImpToRatio
1,South Africa,"Coal; briquettes, ovoids and similar solid fue...",8.131101e+10,8.131101e+10,6.667532e+10,2.059060e+05,1.219507e+00
1186,Nigeria,"Coal; briquettes, ovoids and similar solid fue...",2.682000e+05,2.682000e+05,3.286436e+05,0.000000e+00,8.160815e-01
1194,Egypt,"Coal; briquettes, ovoids and similar solid fue...",1.205169e+08,1.205169e+08,1.162292e+08,0.000000e+00,1.036890e+00
121,South Africa,"Coal; anthracite, whether or not pulverised, b...",1.240604e+09,1.240604e+09,1.110831e+09,2.005220e+05,1.116825e+00
1232,Egypt,"Coal; anthracite, whether or not pulverised, b...",1.203255e+08,1.203255e+08,1.158026e+08,0.000000e+00,1.039057e+00
174,South Africa,"Coal; bituminous, whether or not pulverised, b...",7.892341e+10,7.892341e+10,2.689887e+10,5.384000e+03,2.934079e+00
1259,Egypt,"Coal; bituminous, whether or not pulverised, b...",3.000000e+03,3.000000e+03,1.671700e+05,0.000000e+00,1.794580e-02
242,South Africa,"Coal; (other than anthracite and bituminous), ...",1.016501e+09,1.016501e+09,3.877652e+10,0.000000e+00,2.621435e-02
1266,Nigeria,"Coal; (other than anthracite and bituminous), ...",2.682000e+05,2.682000e+05,2.019835e+05,0.000000e+00,1.327831e+00
1272,Egypt,"Coal; (other than anthracite and bituminous), ...",1.675510e+05,1.675510e+05,2.590240e+05,0.000000e+00,6.468551e-01


In [24]:
# Calculate the means to establish re-imports/re-exports are only small fractions of the total
np.mean(world_sum_exp.ReExportsFrom / world_sum_exp.ExportsFrom)

0.0159435951649826

In [25]:
np.mean(world_sum_imp.ReImportsFrom / world_sum_imp.ImportsFrom)

0.005987563776906325

In [26]:
geomean(world_sum_exp.ExpFromImpToRatio)

0.8262362724448951

In [27]:
geomean_trunc(world_sum_exp.ExpFromImpToRatio)

0.6344662811907632

In [28]:
logstdev_trunc(world_sum_exp.ExpFromImpToRatio)

4.6770147260901895

In [29]:
geomean(world_sum_imp.ImpFromExpToRatio)

38.37993835477144

In [30]:
geomean_trunc(world_sum_imp.ImpFromExpToRatio)

1.4075218191661438

In [31]:
logstdev_trunc(world_sum_imp.ImpFromExpToRatio)

4.708151051689683

In [32]:
# Calculate the fraction of data within 2 orders: 
fraction_within_2_orders_exp = sum(
    (world_sum_exp.ExpFromImpToRatio < 100 ) & (world_sum_exp.ExpFromImpToRatio > 0.01)
)/len(world_sum_exp)
fraction_within_2_orders_exp

0.9047619047619048

In [33]:
fraction_within_2_orders_imp = sum(
    (world_sum_imp.ImpFromExpToRatio < 100 ) & (world_sum_imp.ImpFromExpToRatio > 0.01)
)/len(world_sum_imp)
fraction_within_2_orders_imp

0.8421052631578947

In [34]:
(fraction_within_2_orders_exp * len(world_sum_exp)  + fraction_within_2_orders_imp* len(world_sum_imp))/(len(world_sum_exp)+len(world_sum_imp))

0.8686868686868687

# Begin conversion into emissions equivalents
We use data from https://ghgprotocol.org/calculation-tools#cross_sector_tools_id to perform the conversion. This is in turn based on http://www.ipcc-nggip.iges.or.jp/public/2006gl/vol2.html data, but converted into units of emissions/tonne rather than emissions/TJ. We compare these numbers when applied to coal with the numbers using the USA data, found in https://www.epa.gov/sites/production/files/2020-04/documents/ghg-emission-factors-hub.pdf
to ensure that the values are reasonable. 
## USA analysis

In [35]:
us_folder = "USA_data"
solid_factor_file = "solidFuelsConversionFactors.csv"
solid_factors = pd.read_csv(join(input_folder, us_folder, solid_factor_file), index_col=0)
kgInShortTon = 907.18
solid_factors = solid_factors.append(pd.DataFrame({c: np.nan for c in solid_factors.columns}, index=["nan"]))
solid_factors

,mmBtu per short ton,kg CO2 per mmBtu,g CH4 per mmBtu,g N2O per mmBtu,kg CO2 per short ton,g CH4 per short ton,g N2O per short ton
Anthracite Coal,25.09,103.69,11.0,1.6,2602.0,276.0,40.0
Bituminous Coal,24.93,93.28,11.0,1.6,2325.0,274.0,40.0
Sub-bituminous Coal,17.25,97.17,11.0,1.6,1676.0,190.0,28.0
Lignite Coal,14.21,97.72,11.0,1.6,1389.0,156.0,23.0
Mixed Coal (Commercial Sector),21.39,94.27,11.0,1.6,2016.0,235.0,34.0
Mixed Coal (Electric Power Sector),19.73,95.52,11.0,1.6,1885.0,217.0,32.0
Mixed Coal (Industrial Coking),26.28,93.90,11.0,1.6,2468.0,289.0,42.0
Mixed Coal (Industrial Sector),22.35,94.67,11.0,1.6,2116.0,246.0,36.0
Coal Coke,24.80,113.67,11.0,1.6,2819.0,273.0,40.0
Municipal Solid Waste,9.95,90.70,32.0,4.2,902.0,318.0,42.0


In [36]:
# Work out how these names correspond to the comtrade names
input_from.Commodity.unique()

array(['Coal; briquettes, ovoids and similar solid fuels manufactured from coal',
       'Coal; anthracite, whether or not pulverised, but not agglomerated',
       'Coal; bituminous, whether or not pulverised, but not agglomerated',
       'Coal; (other than anthracite and bituminous), whether or not pulverised but not agglomerated',
       'Briquettes, ovoids and similar solid fuels; manufactured from coal',
       'Lignite; whether or not agglomerated, excluding jet',
       'Peat; (including peat litter), whether or not agglomerated',
       'Coke and semi-coke; of coal, lignite or peat, whether or not agglomerated; retort carbon',
       'Coal gas, water gas, producer gas and similar gases, other than petroleum gases and other gaseous hydrocarbons',
       'Tar distilled from coal, from lignite, peat and other mineral tars, whether or not dehydrated or partially distilled; including reconstituted tars',
       'Oils and other products of the distillation of high temperature coal t

In [37]:
solid_name_conversion = {
    'Coal; anthracite, whether or not pulverised, but not agglomerated': "Anthracite Coal", 
    'Coal; bituminous, whether or not pulverised, but not agglomerated': "Bituminous Coal",
    'Coal; (other than anthracite and bituminous), whether or not pulverised but not agglomerated': "Sub-bituminous Coal",
    'Briquettes, ovoids and similar solid fuels; manufactured from coal': "Mixed Coal (Commercial Sector)",
    'Lignite; whether or not agglomerated, excluding jet': "Lignite Coal",
    'Peat; (including peat litter), whether or not agglomerated': "Peat",
    'Coke and semi-coke; of coal, lignite or peat, whether or not agglomerated; retort carbon': "Coal Coke",
}
world_sum_exp = world_sum_exp
world_sum_exp["emissionsCommodity"] = [
    solid_name_conversion[v]  if v in solid_name_conversion.keys() else "nan" for v in world_sum_exp.Commodity
]
world_sum_imp["emissionsCommodity"] = [
    solid_name_conversion[v]  if v in solid_name_conversion.keys() else "nan" for v in world_sum_imp.Commodity
]
# Apply emissions conversion factors, including converting kg CO2 to tonnes Co2
world_sum_exp["tCO2_USA"] = solid_factors[
    "kg CO2 per short ton"
][world_sum_exp["emissionsCommodity"]].values * world_sum_exp.ExportsFrom / kgInShortTon / 1000 
world_sum_exp["tCH4_USA"] = solid_factors[
    "g CH4 per short ton"
][world_sum_exp["emissionsCommodity"]].values * world_sum_exp.ExportsFrom / kgInShortTon / 1000000
world_sum_exp["tN2O_USA"] = solid_factors[
    "g N2O per short ton"
][world_sum_exp["emissionsCommodity"]].values * world_sum_exp.ExportsFrom / kgInShortTon / 1000000
# And to imports
world_sum_imp["tCO2_USA"] = solid_factors[
    "kg CO2 per short ton"
][world_sum_imp["emissionsCommodity"]].values * world_sum_imp.ImportsFrom / kgInShortTon / 1000 
world_sum_imp["tCH4_USA"] = solid_factors[
    "g CH4 per short ton"
][world_sum_imp["emissionsCommodity"]].values * world_sum_imp.ImportsFrom / kgInShortTon / 1000000
world_sum_imp["tN2O_USA"] = solid_factors[
    "g N2O per short ton"
][world_sum_imp["emissionsCommodity"]].values * world_sum_imp.ImportsFrom / kgInShortTon / 1000000

## GHG protocol analysis

In [38]:
ghgprotocol_folder = "ghg_protocol_UN_data"
co2_file = "CO2_emissions_per_mass_ghgprotocol.csv"
ch4_file = "CH4_emissions_per_mass_ghgprotocol.csv"
n2o_file = "N2O_emissions_per_mass_ghgprotocol.csv"
co2_factors = pd.read_csv(join(input_folder, ghgprotocol_folder, co2_file), index_col=0)
ch4_factors = pd.read_csv(join(input_folder, ghgprotocol_folder, ch4_file), index_col=0)
n2o_factors = pd.read_csv(join(input_folder, ghgprotocol_folder, n2o_file), index_col=0)
co2_factors=co2_factors.append(
    pd.DataFrame({c: 0 for c in co2_factors.columns}, index=["Unburned"])
)
ch4_factors=ch4_factors.append(
    pd.DataFrame({c: 0 for c in ch4_factors.columns}, index=["Unburned"])
)
n2o_factors=n2o_factors.append(
    pd.DataFrame({c: 0 for c in n2o_factors.columns}, index=["Unburned"])
)
ch4_factors

,TJ/Gg,kg/TJ,kg/tonne
Crude oil,42.30,10,0.42300
Orimulsion,27.50,10,0.27500
Natural Gas Liquids,44.20,10,0.44200
Motor gasoline,44.30,10,0.44300
Aviation gasoline,44.30,10,0.44300
Jet gasoline,44.30,10,0.44300
Jet kerosene,44.10,10,0.44100
Other kerosene,43.80,10,0.43800
Shale oil,38.10,10,0.38100
Gas/Diesel oil,43.00,10,0.43000


In [39]:
world_sum_exp.Commodity.unique()

array(['Coal; briquettes, ovoids and similar solid fuels manufactured from coal',
       'Coal; anthracite, whether or not pulverised, but not agglomerated',
       'Coal; bituminous, whether or not pulverised, but not agglomerated',
       'Coal; (other than anthracite and bituminous), whether or not pulverised but not agglomerated',
       'Briquettes, ovoids and similar solid fuels; manufactured from coal',
       'Lignite; whether or not agglomerated, excluding jet',
       'Peat; (including peat litter), whether or not agglomerated',
       'Coke and semi-coke; of coal, lignite or peat, whether or not agglomerated; retort carbon',
       'Coal gas, water gas, producer gas and similar gases, other than petroleum gases and other gaseous hydrocarbons',
       'Tar distilled from coal, from lignite, peat and other mineral tars, whether or not dehydrated or partially distilled; including reconstituted tars',
       'Oils and other products of the distillation of high temperature coal t

In [40]:
ghg_factors_conversion = {
    'Coal; anthracite, whether or not pulverised, but not agglomerated': "Anthracite",
    'Coal; bituminous, whether or not pulverised, but not agglomerated': "Bitumen",
    'Coal; (other than anthracite and bituminous), whether or not pulverised but not agglomerated': "Sub bituminous coal",
    'Briquettes, ovoids and similar solid fuels; manufactured from coal': "Brown coal briquettes",
    'Lignite; whether or not agglomerated, excluding jet': "Lignite",
    'Peat; (including peat litter), whether or not agglomerated': "Peat",
    'Coke and semi-coke; of coal, lignite or peat, whether or not agglomerated; retort carbon': "Coking coal",
    'Coal gas, water gas, producer gas and similar gases, other than petroleum gases and other gaseous hydrocarbons': "Gas coke",
    'Tar distilled from coal, from lignite, peat and other mineral tars, whether or not dehydrated '
    'or partially distilled; including reconstituted tars': "Coal tar",
    'Oils and other products of the distillation of high temperature coal tar; similar products in which the weight '
    'of the aromatic constituents exceeds that of the non-aromatic constituents': "Other petroleum products",
    'Pitch and pitch coke; obtained from coal tar or from other mineral tars': "Coke oven coke",
    'Petroleum oils and oils obtained from bituminous minerals; crude': "Crude oil",
    'Petroleum oils and oils from bituminous minerals, not crude; preparations n.e.c, containing by weight 70% '
    'or more of petroleum oils or oils from bituminous minerals; these being the basic constituents of '
    'the preparations; waste oils': "Motor gasoline",
    'Petroleum gases and other gaseous hydrocarbons': "Natural gas",
    'Petroleum jelly; paraffin wax, micro-crystalline petroleum wax, slack wax, ozokerite, lignite wax, '
    'peat wax, other mineral waxes, similar products obtained by synthesis, other processes; coloured or not': "Unburned",
    'Petroleum coke, petroleum bitumen; other residues of petroleum oils or oils obtained from '
    'bituminous minerals': "Petroleum coke",
    'Bitumen and asphalt, natural; bituminous or oil shale and tar sands; asphaltites '
    'and asphaltic rocks': "Oil shale and tar sands",
    'Bituminous mixtures based on natural asphalt; on natural bitumen, on petroleum bitumen, on mineral '
    'tar or on mineral tar pitch (e.g. bituminous mastics, cut-backs)': "Unburned"
}

In [41]:
world_sum_exp["ghgCategory"] = [
    ghg_factors_conversion[v]  if v in ghg_factors_conversion.keys() else "nan" for v in world_sum_exp.Commodity
]
world_sum_imp["ghgCategory"] = [
    ghg_factors_conversion[v]  if v in ghg_factors_conversion.keys() else "nan" for v in world_sum_imp.Commodity
]
# We want to remove the aggregate values to prevent confusion:
world_sum_exp = world_sum_exp[~world_sum_exp.Commodity.isin(composite)]
world_sum_imp = world_sum_imp[~world_sum_imp.Commodity.isin(composite)]

In [42]:
# Convert from kg product to tonnes, then kg CO2 to tonnes
world_sum_exp["tCO2"] = world_sum_exp.ExportsFrom * co2_factors["kg/tonne"][world_sum_exp.ghgCategory].values / 1000000
world_sum_exp["tCH4"] = world_sum_exp.ExportsFrom * ch4_factors["kg/tonne"][world_sum_exp.ghgCategory].values / 1000000
world_sum_exp["tN2O"] = world_sum_exp.ExportsFrom * n2o_factors["kg/tonne"][world_sum_exp.ghgCategory].values / 1000000
world_sum_exp["tCO2_impto"] = world_sum_exp.ImportsTo * co2_factors["kg/tonne"][world_sum_exp.ghgCategory].values / 1000000
world_sum_exp["tCH4_impto"] = world_sum_exp.ImportsTo * ch4_factors["kg/tonne"][world_sum_exp.ghgCategory].values / 1000000
world_sum_exp["tN2O_impto"] = world_sum_exp.ImportsTo * n2o_factors["kg/tonne"][world_sum_exp.ghgCategory].values / 1000000

world_sum_imp["tCO2"] = world_sum_imp.ImportsFrom * co2_factors["kg/tonne"][world_sum_imp.ghgCategory].values / 1000000
world_sum_imp["tCH4"] = world_sum_imp.ImportsFrom * ch4_factors["kg/tonne"][world_sum_imp.ghgCategory].values / 1000000
world_sum_imp["tN2O"] = world_sum_imp.ImportsFrom * n2o_factors["kg/tonne"][world_sum_imp.ghgCategory].values / 1000000
world_sum_imp["tCO2_expto"] = world_sum_imp.ExportsTo * co2_factors["kg/tonne"][world_sum_imp.ghgCategory].values / 1000000
world_sum_imp["tCH4_expto"] = world_sum_imp.ExportsTo * ch4_factors["kg/tonne"][world_sum_imp.ghgCategory].values / 1000000
world_sum_imp["tN2O_expto"] = world_sum_imp.ExportsTo * n2o_factors["kg/tonne"][world_sum_imp.ghgCategory].values / 1000000
world_sum_imp

c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\ipykernel_launcher.py:4: Settin

,Country,Commodity,ImportsFrom,World,ReImportsFrom,ExportsTo,ImpFromExpToRatio,emissionsCommodity,tCO2_USA,tCH4_USA,tN2O_USA,ghgCategory,tCO2,tCH4,tN2O,tCO2_expto,tCH4_expto,tN2O_expto
1,South Africa,"Coal; anthracite, whether or not pulverised, b...",3.542647e+08,3.542647e+08,0.0,3.681602e+07,9.622568e+00,Anthracite Coal,1.016112e+06,107.781320,15.620481,Anthracite,9.298067e+05,94.588676,1.418830e+01,9.662770e+04,9.829878e+00,1.474482e+00
1,Nigeria,"Coal; anthracite, whether or not pulverised, b...",1.887503e+06,1.887503e+06,0.0,1.687869e+06,1.118276e+00,Anthracite Coal,5.413791e+03,0.574253,0.083225,Anthracite,4.953959e+03,0.503963,7.559450e-02,4.429998e+03,4.506610e-01,6.759915e-02
1,Egypt,"Coal; anthracite, whether or not pulverised, b...",1.269627e+08,8.766593e+07,0.0,2.122845e+07,5.980782e+00,Anthracite Coal,3.641581e+05,38.627074,5.598127,Anthracite,3.332276e+05,33.899045,5.084857e+00,5.571640e+04,5.667996e+00,8.501993e-01
1,South Africa,"Coal; bituminous, whether or not pulverised, b...",1.273978e+09,1.273978e+09,0.0,1.470915e+09,8.661127e-01,Bituminous Coal,3.265062e+06,384.785744,56.173101,Bitumen,4.132963e+06,512.139097,3.072835e+01,4.771853e+06,5.913077e+02,3.547846e+01
1,Nigeria,"Coal; bituminous, whether or not pulverised, b...",0.000000e+00,0.000000e+00,0.0,9.203600e+07,0.000000e+00,Bituminous Coal,0.000000e+00,0.000000,0.000000,Bitumen,0.000000e+00,0.000000,0.000000e+00,2.985777e+05,3.699847e+01,2.219908e+00
1,Egypt,"Coal; bituminous, whether or not pulverised, b...",1.846916e+09,2.715053e+09,0.0,5.545267e+09,3.330617e-01,Bituminous Coal,4.733438e+06,557.833157,81.435497,Bitumen,5.991655e+06,742.460378,4.454762e+01,1.798962e+07,2.229197e+03,1.337518e+02
1,South Africa,"Coal; (other than anthracite and bituminous), ...",1.379110e+09,1.379110e+09,300727000.0,3.501910e+08,3.938164e+00,Sub-bituminous Coal,2.547882e+06,288.841049,42.566049,Sub bituminous coal,2.504863e+06,260.651713,3.909776e+01,6.360483e+05,6.618609e+01,9.927914e+00
1,Nigeria,"Coal; (other than anthracite and bituminous), ...",2.232910e+07,2.232911e+07,0.0,0.000000e+00,2.232910e+27,Sub-bituminous Coal,4.125265e+04,4.676613,0.689185,Sub bituminous coal,4.055613e+04,4.220201,6.330301e-01,0.000000e+00,0.000000e+00,0.000000e+00
1,Egypt,"Coal; (other than anthracite and bituminous), ...",2.260211e+08,2.260211e+08,0.0,3.085194e+07,7.325993e+00,Sub-bituminous Coal,4.175702e+05,47.337910,6.976113,Sub bituminous coal,4.105198e+05,42.717984,6.407698e+00,5.603607e+04,5.831016e+00,8.746524e-01
1,South Africa,"Briquettes, ovoids and similar solid fuels; ma...",4.443221e+07,4.443221e+07,0.0,5.729154e+07,7.755458e-01,Mixed Coal (Commercial Sector),9.874042e+04,11.509920,1.665265,Brown coal briquettes,8.967531e+04,9.197468,1.379620e+00,1.156286e+05,1.185935e+01,1.778902e+00


In [43]:
world_sum_exp.to_csv(join(output_folder, "detailed_exports.csv"))
report_exp = world_sum_exp.groupby(by="Country", axis=0).sum()
report_exp["GWP100 tCO2-equiv"] = report_exp.tCO2 + 28 * report_exp.tCH4 + 265 * report_exp.tN2O
report_exp["GWP100 tCO2-equiv other reporter"] = report_exp.tCO2_impto + 28 * report_exp.tCH4_impto + 265 * report_exp.tN2O_impto
report_exp.to_csv(join(output_folder, "total_exports.csv"))
report_exp

,ExportsFrom,World,ImportsTo,ReExportsFrom,ExpFromImpToRatio,tCO2_USA,tCH4_USA,tN2O_USA,tCO2,tCH4,tN2O,tCO2_impto,tCH4_impto,tN2O_impto,GWP100 tCO2-equiv,GWP100 tCO2-equiv other reporter
Country,,,,,,,,,,,,,,,,
Egypt,1.450403e+10,1.450403e+10,1.692456e+10,0.000000e+00,inf,8.393329e+05,87.196804,12.735905,4.331631e+07,5787.661926,333.680950,5.106953e+07,6726.135158,387.197451,4.356679e+07,5.136046e+07
Nigeria,4.191743e+10,4.415084e+10,1.057870e+11,0.000000e+00,2.689518e+04,5.840877e+02,0.064751,0.009535,1.292993e+08,17448.274942,1031.966973,3.220348e+08,42116.869416,2387.544382,1.300613e+08,3.238468e+08
South Africa,8.741626e+10,8.389265e+10,7.059563e+10,4.982954e+06,2.449856e+01,2.083306e+08,24494.037676,3575.638653,2.795022e+08,34843.896026,2142.188815,1.715636e+08,19894.519190,1887.800000,2.810455e+08,1.726209e+08


In [44]:
world_sum_imp.to_csv(join(output_folder, "detailed_imports.csv"))
report_imp = world_sum_imp.groupby(by="Country", axis=0).sum()
report_imp["GWP100 tCO2-equiv"] = report_imp.tCO2 + 28 * report_imp.tCH4 + 265 * report_imp.tN2O
report_imp["GWP100 tCO2-equiv other reporter"] = report_imp.tCO2_expto + 28 * report_imp.tCH4_expto + 265 * report_imp.tN2O_expto
report_imp.to_csv(join(output_folder, "total_imports.csv"))
report_imp

,ImportsFrom,World,ReImportsFrom,ExportsTo,ImpFromExpToRatio,tCO2_USA,tCH4_USA,tN2O_USA,tCO2,tCH4,tN2O,tCO2_expto,tCH4_expto,tN2O_expto,GWP100 tCO2-equiv,GWP100 tCO2-equiv other reporter
Country,,,,,,,,,,,,,,,,
Egypt,2.809144e+10,2.184741e+10,0.0,1.644957e+10,7.090300e+29,8.028783e+06,889.151554,129.919267,8.336768e+07,10446.518303,616.261592,5.077360e+07,6423.760198,372.374636,8.382349e+07,5.105215e+07
Nigeria,1.198093e+10,9.290448e+09,0.0,1.250538e+10,2.232910e+27,4.855050e+04,5.478367,0.804841,3.660518e+07,5231.401238,314.109716,3.825003e+07,5472.834815,327.180163,3.683490e+07,3.848998e+07
South Africa,3.585824e+10,3.585824e+10,301247673.0,1.494497e+10,1.360238e+03,9.474703e+06,1050.158874,153.496369,1.072664e+08,14147.526447,872.366879,4.495353e+07,5925.958927,375.586294,1.078938e+08,4.521899e+07


In [45]:
report_net_exp = report_exp - report_imp
report_net_exp = report_net_exp[["GWP100 tCO2-equiv", "tCO2", "tCH4", "tN2O", "GWP100 tCO2-equiv other reporter"]]
report_net_exp.to_csv(join(output_folder, "net_exports.csv"))
report_net_exp

,GWP100 tCO2-equiv,tCO2,tCH4,tN2O,GWP100 tCO2-equiv other reporter
Country,,,,,
Egypt,-4.025670e+07,-4.005137e+07,-4658.856378,-282.580642,3.083188e+05
Nigeria,9.322644e+07,9.269413e+07,12216.873704,717.857257,2.853568e+08
South Africa,1.731518e+08,1.722358e+08,20696.369579,1269.821936,1.274019e+08


# Sanity check against Nigeria's oil exports

In [46]:
# Net exports were about 2mil barrels per day: 
# https://ihsmarkit.com/research-analysis/nigerian-crude-oil-exports-to-approach-17-million-bd-in-2020.html
# https://www.ceicdata.com/en/indicator/nigeria/crude-oil-exports
ReportedNigerianOilExport = 140*2000000*365
# And net imports is around 20 billion litres: https://www.statista.com/statistics/1165962/petrol-import-in-nigeria/
ReportedNigerianNetOilImport = 20000000000

In [47]:
ReportedNigerianOilExport

102200000000

In [48]:
ReportedNigerianNetOilImport/ReportedNigerianOilExport

0.19569471624266144

In [49]:
oils = ["Crude oil", "Motor gasoline", "Other petroleum products", "Oil shale and tar sands", "Petroleum coke"]
NigerianOilExp = world_sum_exp[(world_sum_exp.Country=="Nigeria") & (world_sum_exp.ghgCategory.isin(
    oils
))]
NigerianOilExp

,Country,Commodity,ExportsFrom,World,ImportsTo,ReExportsFrom,ExpFromImpToRatio,emissionsCommodity,tCO2_USA,tCH4_USA,tN2O_USA,ghgCategory,tCO2,tCH4,tN2O,tCO2_impto,tCH4_impto,tN2O_impto
1375,Nigeria,Oils and other products of the distillation of...,1.703527e+08,1.703527e+08,6.334539e+03,0.0,26892.673262,nan,NaN,NaN,NaN,Other petroleum products,5.019715e+05,68.481785,4.108907,1.866573e+01,0.002546,0.000153
1424,Nigeria,Petroleum oils and oils obtained from bitumino...,3.991407e+10,3.991407e+10,8.987151e+10,0.0,0.444124,nan,NaN,NaN,NaN,Crude oil,1.237572e+08,16883.653443,1013.019207,2.786547e+08,38015.648730,2280.938924
1478,Nigeria,Petroleum oils and oils from bituminous minera...,2.769994e+08,2.769994e+08,1.386770e+09,0.0,0.199744,nan,NaN,NaN,NaN,Motor gasoline,8.503854e+05,122.710743,7.362645,4.257369e+06,614.339020,36.860341


In [50]:
NigerianOilImp = world_sum_imp[(world_sum_imp.Country=="Nigeria") & (world_sum_imp.ghgCategory.isin(
    oils
))]
NigerianOilImp

,Country,Commodity,ImportsFrom,World,ReImportsFrom,ExportsTo,ImpFromExpToRatio,emissionsCommodity,tCO2_USA,tCH4_USA,tN2O_USA,ghgCategory,tCO2,tCH4,tN2O,tCO2_expto,tCH4_expto,tN2O_expto
1,Nigeria,Oils and other products of the distillation of...,3.818150e+06,3.818151e+06,0.0,2.812205e+06,1.357707,nan,NaN,NaN,NaN,Other petroleum products,1.125079e+04,1.534896,0.092094,8.286612e+03,1.130506,0.067830
1,Nigeria,Petroleum oils and oils obtained from bitumino...,0.000000e+00,0.000000e+00,0.0,4.012510e+05,0.000000,nan,NaN,NaN,NaN,Crude oil,0.000000e+00,0.000000,0.000000,1.244115e+03,0.169729,0.010184
1,Nigeria,Petroleum oils and oils from bituminous minera...,1.153075e+10,8.867722e+09,0.0,1.207998e+10,0.954534,nan,NaN,NaN,NaN,Motor gasoline,3.539930e+07,5108.123737,306.487424,3.708542e+07,5351.431627,321.085898
1,Nigeria,"Petroleum coke, petroleum bitumen; other resid...",3.422620e+08,3.181909e+08,0.0,1.566926e+08,2.184290,nan,NaN,NaN,NaN,Petroleum coke,1.084543e+06,111.235148,6.674109,4.965196e+05,50.925087,3.055505
1,Nigeria,"Bitumen and asphalt, natural; bituminous or oi...",8.215250e+05,8.215260e+05,0.0,2.834900e+05,2.897898,nan,NaN,NaN,NaN,Oil shale and tar sands,7.823383e+02,0.073116,0.010967,2.699675e+02,0.025231,0.003785


The sanity check indicates that the difference of the imports and exports is more robust than either individually, possibly due to transport of oil through Nigeria

In [51]:
copy_from = from_year[from_year.Commodity==crude]
copy_from["ratio"] = copy_from[tv] / copy_from[nw] 
copy_from[copy_from.Reporter == "Nigeria"]

c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Year,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,2nd Partner Code,2nd Partner,2nd Partner ISO,Commodity Code,Commodity,Netweight (kg),Trade Value (US$),Flag,Infilled,ratio
1424,2018,4,0,2,Export,Nigeria,NGA,0,World,WLD,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,3.991407e+10,51371239561,6,True,1.287046
1425,2018,4,0,2,Export,Nigeria,NGA,32,Argentina,ARG,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,9.943778e+07,127980973,6,True,1.287046
1426,2018,4,0,2,Export,Nigeria,NGA,36,Australia,AUS,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,1.985002e+08,255478808,6,True,1.287046
1427,2018,4,0,2,Export,Nigeria,NGA,76,Brazil,BRA,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,1.372792e+09,1766845609,6,True,1.287046
1428,2018,4,0,2,Export,Nigeria,NGA,120,Cameroon,CMR,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,2.066078e+08,265913670,6,True,1.287046
1429,2018,4,0,2,Export,Nigeria,NGA,124,Canada,CAN,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,1.347342e+09,1734090341,6,True,1.287046
1430,2018,4,0,2,Export,Nigeria,NGA,156,China,CHN,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,1.218297e+08,156800444,6,True,1.287046
1431,2018,4,0,2,Export,Nigeria,NGA,214,Dominican Rep.,DOM,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,2.839938e+08,365513054,6,True,1.287046
1432,2018,4,0,2,Export,Nigeria,NGA,251,France,FRA,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,3.121801e+09,4017900091,6,True,1.287046
1433,2018,4,0,2,Export,Nigeria,NGA,276,Germany,DEU,NaN,NaN,NaN,2709,Petroleum oils and oils obtained from bitumino...,9.687036e+08,1246765848,6,True,1.287046


In [52]:
sum(NigerianOilExp.ExportsFrom) / ReportedNigerianOilExport

0.39492589483890195

In [53]:
sum(NigerianOilExp.ImportsTo) / ReportedNigerianOilExport

0.8929382204656542

In [54]:
sum(NigerianOilImp.ImportsFrom) / ReportedNigerianNetOilImport

0.593882751288667

In [55]:
sum(NigerianOilImp.ExportsTo) / ReportedNigerianNetOilImport

0.612008530998762

## Sectoral fractions
Convert the different emissions categories into proportions for import, export and net

In [56]:
coals = ['Anthracite', 'Bitumen', 'Sub bituminous coal', 'Brown coal briquettes', 'Coke oven coke',
       'Other bituminous coal', 'Lignite', 'Peat', 'Coking coal', 'Coal tar', 'Petroleum coke']
gases = ["Natural gas", "Gas coke"]
all_burned_cats = coals + gases + oils

In [57]:
assert [v for v in set(
     world_sum_imp.ghgCategory.append(world_sum_exp.ghgCategory)
 ) if v not in all_burned_cats] == ["Unburned"]

In [58]:
indexed_world_imp = world_sum_imp.copy().set_index(["Country", "ghgCategory"], drop=True)
indexed_world_exp = world_sum_exp.copy().set_index(["Country", "ghgCategory"], drop=True)
world_net_exp = pd.DataFrame(columns=["tCO2"], index=set(
    indexed_world_exp.index.append(indexed_world_imp.index)
))
for ind in world_net_exp.index:
    tmp = 0
    try:
        tmp += indexed_world_imp["tCO2"][ind][0]
    except KeyError:
        pass
    try:
        tmp += indexed_world_exp["tCO2"][ind][0]
    except KeyError:
        pass
    world_net_exp["tCO2"][ind] = tmp
world_net_exp

c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\IPython\core\interactiveshell.py:2877: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\IPython\core\async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


tCO2
South Africa Coke oven coke                 115603
             Peat                          59129.1
Egypt        Motor gasoline            5.07626e+07
Nigeria      Anthracite                    4953.96
South Africa Brown coal briquettes          353044
             Coal tar                      3600.58
             Crude oil                 6.28675e+07
Egypt        Petroleum coke            1.13138e+07
Nigeria      Petroleum coke            1.08454e+06
             Oil shale and tar sands       782.338
Egypt        Other petroleum products      40536.4
Nigeria      Sub bituminous coal           41043.3
             Lignite                      0.161501
Egypt        Unburned                            0
Nigeria      Coking coal                   1365.61
             Natural gas               4.25133e+06
South Africa Motor gasoline            4.12992e+07
Egypt        Gas coke                     0.181044
South Africa Unburned                            0
             Bitumen                   2.60172e+08
             Gas coke                      64.7655
Egypt        Coking coal               2.42113e+06
             Brown coal briquettes         1070.32
             Crude oil                 3.63864e+07
             Natural gas               1.79005e+07
South Africa Petroleum coke            1.15278e+06
Nigeria      Coke oven coke                      0
Egypt        Oil shale and tar sands        550023
South Africa Anthracite                4.18591e+06
Nigeria      Coal tar                            0
South Africa Other petroleum products       793488
Egypt        Lignite                        138212
             Anthracite                     649035
Nigeria      Crude oil                 1.23757e+08
             Motor gasoline            3.62497e+07
             Brown coal briquettes         145.677
Egypt        Coke oven coke                96536.3
South Africa Oil shale and tar sands       23325.3
Nigeria      Peat                          232.109
South Africa Lignite                      0.176679
Egypt        Coal tar                      11188.4
South Africa Coking coal               2.42354e+06
             Natural gas                8.9688e+06
             Sub bituminous coal       4.35112e+06
Egypt        Bitumen                   5.99166e+06
Nigeria      Bitumen                             0
             Gas coke                      13.5783
Egypt        Sub bituminous coal            410824
Nigeria      Other petroleum products       513222
             Unburned                            0
Egypt        Peat                          10552.5

In [59]:
fraction_table = pd.DataFrame(columns=["ImpExp", "Class", "Subclass"] + countries)
for impexp in ["Import", "Export", "Net export"]:
    use_table = indexed_world_imp if impexp == "Import" else indexed_world_exp if impexp == "Export" else world_net_exp
    for item in all_burned_cats:
        country_item_dict = {}
        for country in countries:
            try:
                val = use_table["tCO2"].loc[(country, item)]/sum(use_table["tCO2"].loc[use_table.index.get_level_values(0)==country])*100
                if impexp != "Net export":
                    val = val[0]
                country_item_dict[country] = "{:.2E}%".format(val)
            except KeyError:
                pass
        if country_item_dict:
            base_dict = {
                "ImpExp": impexp,
                "Class": "Coal" if item in coals else "Oil" if item in oils else "Gas",
                "Subclass": item
            }
            base_dict.update(country_item_dict)
            fraction_table = fraction_table.append(pd.DataFrame(base_dict, index=[1]))
    # Now we append a line with the total values
    base_dict = {
                "ImpExp": impexp,
                "Class": "Total CO2",
                "Subclass": ""
            }
    country_item_dict = {}
    for country in countries:
        country_item_dict[country] = sum(use_table["tCO2"].loc[use_table.index.get_level_values(0)==country])
    base_dict.update(country_item_dict)
    fraction_table = fraction_table.append(pd.DataFrame(base_dict, index=[2]))
fraction_table.to_csv(join(output_folder, "fraction_of_co2.csv"))
fraction_table

c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\ipykernel_launcher.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  


,ImpExp,Class,Subclass,South Africa,Nigeria,Egypt
1,Import,Coal,Anthracite,8.67E-01%,1.35E-02%,4.00E-01%
1,Import,Coal,Bitumen,3.85E+00%,0.00E+00%,7.19E+00%
1,Import,Coal,Sub bituminous coal,2.34E+00%,1.11E-01%,4.92E-01%
1,Import,Coal,Brown coal briquettes,8.36E-02%,3.98E-04%,1.23E-03%
1,Import,Coal,Coke oven coke,1.02E-01%,0.00E+00%,1.16E-01%
1,Import,Coal,Lignite,1.51E-07%,4.41E-07%,9.27E-05%
1,Import,Coal,Peat,5.47E-02%,6.34E-04%,1.27E-02%
1,Import,Coal,Coking coal,1.99E+00%,3.52E-03%,2.58E+00%
1,Import,Coal,Coal tar,2.57E-03%,0.00E+00%,1.22E-05%
1,Import,Coal,Petroleum coke,8.09E-01%,2.96E+00%,1.35E+01%
